# Transfer Learning

**Exercise objetives**
- Use a pretrained neural network : Transfer learning

#### Transfer what?
We have seen that the convolutions are mathematical operation that detect specific patterns in input images and use them to classify the image. One could imagine that these patterns are not 100% specific to the task but to the input images. 

💡 Therefore, **why not using convolutions that have been learnt on other task** with the expectation that it will also work in other scenario?  We _transfer_ a CNN from one task to another => _transfer learning_. This has two advantages:
- taking less time to train
- benefiting from complex architecture that have been trained for state-of-the-art challenges. 

⚠️ Although convolutions may not be specific, the last layer is by design specific to the problem it was trained on! Therefore, this last layer is usually removed, replace by a layer that is design to the task. As this new last layer has random weight, it has to be retrained. This is called _fine-tunning_. 

#### VGG16
In this exercise, we will use the [VGG-16 Neural Network](https://neurohive.io/en/popular-networks/vgg16/), a well-known architecture that has been trained on ImageNet which is a very large database of images of different categories. In a nutshell, this architecture has already learnt kernels which are supposed to be good not only for the task it has been train on but maybe for other tasks. 

<center><img src="https://neurohive.io/wp-content/uploads/2018/11/vgg16-1-e1542731207177.png" width=400></center>

The idea is that first layers are not specialized for the particular task it has been trained on ; only the last ones are. Therefore, we will 
- load the existing VGG16 network
- remove the last fully connected layers
- replace them by new connected layers (whose weights are randomly set)
- and train these last layers on a specific classification task - here, separate types of flower. 

The underlying idea is that the first convolutional layers of VGG-16, that has already been trained, corresponds to filters that are able to extract meaning features from images. And you will only learn the last layers for your particular problem.

<hr>
<hr>

# 1. Data loading & Preprocessing

You have two options to load the data on Google Colab.


**Option 1: Loading the data directly**

You can first get the data onto google Colab thanks to:

`!wget https://wagon-public-datasets.s3.amazonaws.com/flowers-dataset.zip`,

and then run 

`!unzip flowers-dataset.zip`

This is a very easy option to load the data into your working directory.

**Option 2: Adding the data to Google Drive.**

You can first download the data  from `https://wagon-public-datasets.s3.amazonaws.com/flowers-dataset.zip`. Then you have to add them to your Google Drive in a folder called `Deep_learning_data` (for instance) and run the following code in the notebook.: 

```
from google.colab import drive
drive.mount('/content/drive/')
```

The previous code will ask you to go to a given webpage where you copy the link and past it in the Colab form that will appear. Do so to load the data on Google Colab.

Why choosing this option over the first one? This can be of interest if you work in a project team, and update the data from time to time. By doing this, you can share the same data folder within a team, and be sure that everyone has the same at any time, even though someone changes it. The drawback is that Google Colab has now access to your Google Folder, which you might not be or not in favor of, depending on your sensibility.


❓ **Question** ❓ Use one of the above method to load your data

In [1]:
option_1 = True # Choose here

if option_1:
    !wget https://wagon-public-datasets.s3.amazonaws.com/flowers-dataset.zip
    !unzip flowers-dataset.zip
else:
    from google.colab import drive
    drive.mount('/content/drive/')

--2021-11-10 16:06:20--  https://wagon-public-datasets.s3.amazonaws.com/flowers-dataset.zip
Resolving wagon-public-datasets.s3.amazonaws.com (wagon-public-datasets.s3.amazonaws.com)... 52.218.116.185
Connecting to wagon-public-datasets.s3.amazonaws.com (wagon-public-datasets.s3.amazonaws.com)|52.218.116.185|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 104983809 (100M) [application/zip]
Saving to: ‘flowers-dataset.zip’

flowers-dataset.zip 100%[===================>] 100.12M  4.18MB/s    in 39s     

2021-11-10 16:06:59 (2.59 MB/s) - ‘flowers-dataset.zip’ saved [104983809/104983809]

Archive:  flowers-dataset.zip
  inflating: flowers/daisy/2607132536_d95198e619_n.jpg  
  inflating: flowers/daisy/6480809771_b1e14c5cc2_m.jpg  
  inflating: flowers/daisy/9350942387_5b1d043c26_n.jpg  
  inflating: flowers/daisy/4065883015_4bb6010cb7_n.jpg  
  inflating: flowers/daisy/7924174040_444d5bbb8a.jpg  
  inflating: flowers/daisy/8740807508_0587f5b7b7.jpg  
  inflating: f

  inflating: flowers/dandelion/14614655810_9910e6dbd6_n.jpg  
  inflating: flowers/rose/488849503_63a290a8c2_m.jpg  
  inflating: flowers/daisy/13977181862_f8237b6b52.jpg  
  inflating: flowers/daisy/33814092924_b23d019011_n.jpg  
  inflating: flowers/daisy/34342014230_4230ae8e08_n.jpg  
  inflating: flowers/dandelion/5628515159_6b437ff1e5_n.jpg  
  inflating: flowers/daisy/19019544592_b64469bf84_n.jpg  
  inflating: flowers/dandelion/19437710780_c5f2156438.jpg  
  inflating: flowers/rose/5002452394_22896fb9df_n.jpg  
  inflating: flowers/daisy/10994032453_ac7f8d9e2e.jpg  
  inflating: flowers/daisy/25360380_1a881a5648.jpg  
  inflating: flowers/dandelion/7315832212_b0ceeb8de8_n.jpg  
  inflating: flowers/daisy/6208851904_9d916ebb32_n.jpg  
  inflating: flowers/dandelion/9719816995_8f211abf02_n.jpg  
  inflating: flowers/daisy/3764116502_f394428ee0_n.jpg  
  inflating: flowers/daisy/2561352120_7961d8263f.jpg  
  inflating: flowers/daisy/12585131704_0f64b17059_m.jpg  
  inflating: flowe

  inflating: flowers/daisy/5997702776_c7bc37aa6b_n.jpg  
  inflating: flowers/daisy/14674743211_f68b13f6d9.jpg  
  inflating: flowers/daisy/9225336602_e6c392f941_n.jpg  
  inflating: flowers/daisy/3356112863_75da8bca2c_m.jpg  
  inflating: flowers/dandelion/4696437766_85952d0196.jpg  
  inflating: flowers/dandelion/4265711814_9a006ee5b8.jpg  
  inflating: flowers/daisy/3611577717_f3a7a8c416_n.jpg  
  inflating: flowers/dandelion/34694280496_d457f0d7b7_n.jpg  
  inflating: flowers/rose/1562198683_8cd8cb5876_n.jpg  
  inflating: flowers/rose/3948347096_42261f047a_m.jpg  
  inflating: flowers/daisy/9467543719_c4800becbb_m.jpg  
  inflating: flowers/dandelion/5772194932_60b833091f.jpg  
  inflating: flowers/daisy/34326606950_41ff8997d7_n.jpg  
  inflating: flowers/dandelion/16970837587_4a9d8500d7.jpg  
  inflating: flowers/dandelion/5003160931_cf8cbb846f.jpg  
  inflating: flowers/rose/5002458392_f465ed905e_n.jpg  
  inflating: flowers/dandelion/34694292346_83b4d97809_n.jpg  
  inflating: 

  inflating: flowers/dandelion/15358221063_2c6e548e84.jpg  
  inflating: flowers/rose/5001847693_77a9fb4f3c_n.jpg  
  inflating: flowers/dandelion/921252114_91e334b950.jpg  
  inflating: flowers/daisy/2617111535_54c2ac8462.jpg  
  inflating: flowers/dandelion/5749815755_12f9214649_n.jpg  
  inflating: flowers/dandelion/4151883194_e45505934d_n.jpg  
  inflating: flowers/dandelion/3512879565_88dd8fc269_n.jpg  
  inflating: flowers/daisy/5869147563_66fb88119d.jpg  
  inflating: flowers/daisy/34701078235_4a770d14a1_n.jpg  
  inflating: flowers/daisy/102841525_bd6628ae3c.jpg  
  inflating: flowers/daisy/33871813063_d7b6bedafe_n.jpg  
  inflating: flowers/daisy/4432271543_01c56ca3a9.jpg  
  inflating: flowers/dandelion/3589816063_50f8de7b64_m.jpg  
  inflating: flowers/daisy/3552074420_2a0a7166db_m.jpg  
  inflating: flowers/dandelion/8270191872_61e47ae3b8_m.jpg  
  inflating: flowers/daisy/5626784099_b36dd3fb11_n.jpg  
  inflating: flowers/dandelion/14093789753_f0f1acdb57.jpg  
  inflating:

  inflating: flowers/daisy/510844526_858b8fe4db.jpg  
  inflating: flowers/daisy/4281102584_c548a69b81_m.jpg  
  inflating: flowers/daisy/7630520686_e3a61ac763.jpg  
  inflating: flowers/daisy/2481823240_eab0d86921.jpg  
  inflating: flowers/daisy/33843400403_db00aa16b8_n.jpg  
  inflating: flowers/dandelion/34587720941_ccbbc420ec_n.jpg  
  inflating: flowers/dandelion/6897671808_57230e04c5_n.jpg  
  inflating: flowers/daisy/4511693548_20f9bd2b9c_m.jpg  
  inflating: flowers/dandelion/3419172904_7708414ae9_n.jpg  
  inflating: flowers/daisy/15813862117_dedcd1c56f_m.jpg  
  inflating: flowers/daisy/4278442064_a5a598524b_m.jpg  
  inflating: flowers/rose/2677417735_a697052d2d_n.jpg  
  inflating: flowers/dandelion/17862580326_293070978d_m.jpg  
  inflating: flowers/daisy/14707111433_cce08ee007.jpg  
  inflating: flowers/daisy/34718882165_68cdc9def9_n.jpg  
  inflating: flowers/dandelion/34311470786_5b2b5bea1d_n.jpg  
  inflating: flowers/daisy/4258408909_b7cc92741c_m.jpg  
  inflating: f

...

  inflating: flowers/dandelion/3454102259_957ecd0a9b.jpg  
  inflating: flowers/dandelion/34578947551_863af3acb3_n.jpg  
  inflating: flowers/daisy/2838487505_6c3b48efa5_m.jpg  
  inflating: flowers/daisy/6046940312_8faf552f3e_n.jpg  
  inflating: flowers/dandelion/34653465656_31bc613631_n.jpg  
  inflating: flowers/daisy/1140299375_3aa7024466.jpg  
  inflating: flowers/daisy/10140303196_b88d3d6cec.jpg  
  inflating: flowers/rose/4764674741_82b8f93359_n.jpg  
  inflating: flowers/dandelion/6954604340_d3223ed296_m.jpg  
  inflating: flowers/daisy/21402054779_759366efb0_n.jpg  
  inflating: flowers/daisy/16492248512_61a57dfec1_m.jpg  
  inflating: flowers/daisy/1396526833_fb867165be_n.jpg  
  inflating: flowers/dandelion/7062171343_db61c92737_n.jpg  
  inflating: flowers/daisy/9515186037_3be48fe68f.jpg  
  inflating: flowers/rose/2609353769_dc3654f12f.jpg  
  inflating: flowers/daisy/15306268004_4680ba95e1.jpg  
  inflating: flowers/dandelion/19599413676_fc9ee2640e.jpg  
  inflating: flo

  inflating: flowers/daisy/4666648087_b10f376f19.jpg  
  inflating: flowers/rose/3873271620_1d9d314f01_n.jpg  
  inflating: flowers/rose/5172171681_5934378f08.jpg  
  inflating: flowers/dandelion/461632542_0387557eff.jpg  
  inflating: flowers/dandelion/2326334426_2dc74fceb1.jpg  
  inflating: flowers/dandelion/155646858_9a8b5e8fc8.jpg  
  inflating: flowers/dandelion/4574102507_70039c8b28.jpg  
  inflating: flowers/dandelion/4714026966_93846ddb74_m.jpg  
  inflating: flowers/daisy/7066602021_2647457985_m.jpg  
  inflating: flowers/daisy/33857335804_7f0c96243b_n.jpg  
  inflating: flowers/daisy/4085794721_7cd88e0a6c_m.jpg  
  inflating: flowers/daisy/34695914906_961f92ffcd_n.jpg  
  inflating: flowers/dandelion/34689593326_0fd3fbc38a_n.jpg  
  inflating: flowers/dandelion/16650892835_9228a3ef67_m.jpg  
  inflating: flowers/dandelion/18001393975_2a6acaabd8.jpg  
  inflating: flowers/dandelion/5416388641_c66d52d2ff_m.jpg  
  inflating: flowers/rose/323872063_7264e7e018_m.jpg  
  inflatin

  inflating: flowers/rose/4675532860_890504a4a3_m.jpg  
  inflating: flowers/dandelion/34722644565_812fcc1397_n.jpg  
  inflating: flowers/dandelion/451965300_619b781dc9_m.jpg  
  inflating: flowers/dandelion/7132605107_f5e033d725_n.jpg  
  inflating: flowers/dandelion/5829610661_8439ba4a77_n.jpg  
  inflating: flowers/dandelion/26741270544_f44f3a1b19_n.jpg  
  inflating: flowers/dandelion/6012046444_fd80afb63a_n.jpg  
  inflating: flowers/dandelion/3502447188_ab4a5055ac_m.jpg  
  inflating: flowers/rose/5002447916_8a7b7b689a_n.jpg  
  inflating: flowers/dandelion/34256868592_ca3ffd6b15_n.jpg  
  inflating: flowers/dandelion/2598486434_bf349854f2_m.jpg  
  inflating: flowers/dandelion/18089878729_907ed2c7cd_m.jpg  
  inflating: flowers/daisy/34701198765_54aa641d7a_n.jpg  
  inflating: flowers/dandelion/138132145_782763b84f_m.jpg  
  inflating: flowers/rose/5002457330_b77672da1c_n.jpg  
  inflating: flowers/dandelion/10946896405_81d2d50941_m.jpg  
  inflating: flowers/daisy/34733243845_

  inflating: flowers/dandelion/144686365_d7e96941ee_n.jpg  
  inflating: flowers/rose/3292434691_392071d702_n.jpg  
  inflating: flowers/dandelion/7262863194_682209e9fb.jpg  
  inflating: flowers/daisy/4565255237_9ba29c4d4e_n.jpg  
  inflating: flowers/daisy/5924910021_b6debeb7b5_n.jpg  
  inflating: flowers/daisy/3533954656_79156c8473.jpg  
  inflating: flowers/dandelion/4607183665_3472643bc8.jpg  
  inflating: flowers/dandelion/33878004304_a7c823115b_n.jpg  
  inflating: flowers/rose/5159317458_bbb22e2f65_n.jpg  
  inflating: flowers/daisy/6529588249_d9cbe68aab_n.jpg  
  inflating: flowers/rose/2675221506_5286c0595f.jpg  
  inflating: flowers/daisy/20619292635_9857a12d54.jpg  
  inflating: flowers/dandelion/3372748508_e5a4eacfcb_n.jpg  
  inflating: flowers/dandelion/5024965767_230f140d60_n.jpg  
  inflating: flowers/dandelion/6994933428_307b092ce7_m.jpg  
  inflating: flowers/dandelion/2597655841_07fb2955a4.jpg  
  inflating: flowers/daisy/476856232_7c35952f40_n.jpg  
  inflating: f

  inflating: flowers/dandelion/578938011_34918b1468.jpg  
  inflating: flowers/daisy/18582579815_4c6637e9ff_m.jpg  
  inflating: flowers/dandelion/16375088191_2bf2916b53.jpg  
  inflating: flowers/daisy/21805938544_bf6bb0e4bc.jpg  
  inflating: flowers/daisy/14551098743_2842e7a004_n.jpg  
  inflating: flowers/daisy/8938566373_d129e7af75.jpg  
  inflating: flowers/dandelion/34335432530_cd667cbb19_n.jpg  
  inflating: flowers/dandelion/14060367700_fe87e99b6a_m.jpg  
  inflating: flowers/dandelion/7197581386_8a51f1bb12_n.jpg  
  inflating: flowers/daisy/8063844363_db3f4dea85.jpg  
  inflating: flowers/daisy/1031799732_e7f4008c03.jpg  
  inflating: flowers/dandelion/5727534342_419604c177_n.jpg  
  inflating: flowers/rose/218630974_5646dafc63_m.jpg  
  inflating: flowers/dandelion/455728598_c5f3e7fc71_m.jpg  
  inflating: flowers/daisy/12701063955_4840594ea6_n.jpg  
  inflating: flowers/daisy/3410906335_37e8a24b1c_n.jpg  
  inflating: flowers/daisy/2408024540_37f0be7cc0_n.jpg  
  inflating:

❓ **Question** ❓ Use the following method to create 
`X_train, y_train, X_val, y_val, X_test, y_test, num_classes` depending on the `loading_method` you have used

In [2]:
from tensorflow.keras.utils import to_categorical
from tqdm import tqdm
import numpy as np
import os
from PIL import Image

def load_flowers_data(loading_method):
    if loading_method == 'colab':
        data_path = '/content/drive/My Drive/Deep_learning_data/flowers'
    elif loading_method == 'direct':
        data_path = 'flowers/'
    classes = {'daisy':0, 'dandelion':1, 'rose':2}
    imgs = []
    labels = []
    for (cl, i) in classes.items():
        images_path = [elt for elt in os.listdir(os.path.join(data_path, cl)) if elt.find('.jpg')>0]
        for img in tqdm(images_path[:300]):
            path = os.path.join(data_path, cl, img)
            if os.path.exists(path):
                image = Image.open(path)
                image = image.resize((256, 256))
                imgs.append(np.array(image))
                labels.append(i)

    X = np.array(imgs)
    num_classes = len(set(labels))
    y = to_categorical(labels, num_classes)

    # Finally we shuffle:
    p = np.random.permutation(len(X))
    X, y = X[p], y[p]

    first_split = int(len(imgs) /6.)
    second_split = first_split + int(len(imgs) * 0.2)
    X_test, X_val, X_train = X[:first_split], X[first_split:second_split], X[second_split:]
    y_test, y_val, y_train = y[:first_split], y[first_split:second_split], y[second_split:]
    
    return X_train, y_train, X_val, y_val, X_test, y_test, num_classes

In [3]:
# CALL load_flowers_data WITH YOUR PREFERRED METHOD HERE
X_train, y_train, X_val, y_val, X_test, y_test, num_classes = load_flowers_data('direct')

100%|███████████████████████████████████████████████████████████████████████████| 299/299 [00:02<00:00, 127.06it/s]


❓ Check image shape and plot few of them

In [4]:
from matplotlib import pyplot as plt
plt.imshow(X_train[21])

# 2. Home made model

First, let's check our performance on a home-made CNN model

❓ **Questions** ❓ 

- Build, compile and fit a CNN model adapted to the challenge.
- Compare performance with baseline 
- We recommand to use the following architecture

---
```python
from tensorflow.keras.layers.experimental.preprocessing import Rescaling

model = Sequential()

# Notice this cool new layer that "pipe" your rescaling within the architecture
model.add(Rescaling(1./255, input_shape=(256,256,3)))

# Lets add 3 convolution layers, with relatively large kernel size as our pictures are quite big too
model.add(layers.Conv2D(16, kernel_size=10, activation='relu'))
model.add(layers.MaxPooling2D(3))

model.add(layers.Conv2D(32, kernel_size=8, activation="relu"))
model.add(layers.MaxPooling2D(3))

model.add(layers.Conv2D(32, kernel_size=6, activation="relu"))
model.add(layers.MaxPooling2D(3))

model.add(layers.Flatten())
model.add(layers.Dense(100, activation='relu'))
model.add(layers.Dense(3, activation='softmax'))
```
---

In [6]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Reshape
from tensorflow.keras import layers
from tensorflow.keras import models, callbacks

In [7]:
# YOUR CODE HERE
from tensorflow.keras.layers.experimental.preprocessing import Rescaling
model = Sequential()

# Notice this cool new layer that "pipe" your rescaling within the architecture
model.add(Rescaling(1./255, input_shape=(256,256,3)))

# Lets add 3 convolution layers, with relatively large kernel size as our pictures are quite big too
model.add(layers.Conv2D(16, kernel_size=10, activation='relu'))
model.add(layers.MaxPooling2D(3))

model.add(layers.Conv2D(32, kernel_size=8, activation="relu"))
model.add(layers.MaxPooling2D(3))

model.add(layers.Conv2D(32, kernel_size=6, activation="relu"))
model.add(layers.MaxPooling2D(3))

model.add(layers.Flatten())
model.add(layers.Dense(100, activation='relu'))
model.add(layers.Dense(3, activation='softmax'))

es = callbacks.EarlyStopping(patience=30, restore_best_weights=True)
model = Sequential()
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

2021-11-10 16:08:08.806231: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [8]:
from tensorflow.keras import models, callbacks
model.fit(X_train, y_train,
          epochs=200,  # Use early stop in practice
          batch_size=32, 
          validation_split=0.2,
          callbacks=[es]) 
    # YOUR CODE HERE
model.summary() 

Epoch 1/200


2021-11-10 16:08:31.007843: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)


ValueError: in user code:

    /Users/selmalopez/.pyenv/versions/3.8.12/envs/lewagon_current/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:850 train_function  *
        return step_function(self, iterator)
    /Users/selmalopez/.pyenv/versions/3.8.12/envs/lewagon_current/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:840 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /Users/selmalopez/.pyenv/versions/3.8.12/envs/lewagon_current/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:1285 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /Users/selmalopez/.pyenv/versions/3.8.12/envs/lewagon_current/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2833 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /Users/selmalopez/.pyenv/versions/3.8.12/envs/lewagon_current/lib/python3

# 3. Transfer learning

## 3.1 Load VGG16 model


❓ **Question** ❓ Write a first function `load_model()` that loads the pretrained VGG-16 model from `tensorflow.keras.applications.vgg16`. Especially, look at the [documentation](https://www.tensorflow.org/api_docs/python/tf/keras/applications/VGG16) to load the model where:
- the `weights` have been learnt on `imagenet`
- the `input_shape` corresponds to the input shape of any of your images - you have to resize them in case they are not of the same size
- the `include_top` argument is set to `False` in order not to load the fully-connected layers of the VGG-16 without the last layer which was specifically trained on `imagenet`

❗ **Remark** ❗ Do not change the default value of the other arguments

In [0]:
from tensorflow.keras.applications.vgg16 import VGG16

def load_model():
    pass  # YOUR CODE HERE
    
    return model
    

❓ **Question** ❓ Look at the architecture of the model thanks to the summary method

In [0]:
# YOUR CODE HERE

<img src="https://neurohive.io/wp-content/uploads/2018/11/vgg16-1-e1542731207177.png">

Impressive, right? Two things to notice:
- It ends with a convolution layer (namely a maxpooling layer that is the layer that follows a convolution). The flattening of the output and the fully connected layers are not here yet! We need to add them !
- There are more than 14.000.000 parameters, which is a lot. We could fine-tune them, meaning update them as we will update the last layers weights, but it will take a lot of time. For that reason, we will inform the model that the layers until the flattening are non-trainable.

❓ **Question** ❓ Write a first function that takes the previous model as input the set the first layers to be non-trainable, by applying `model.trainable = False`. Then check-out the summary of the model to see that now, the parameters are `non-trainable`



In [0]:
def set_nontrainable_layers(model):
    # Set the first layers to be untrainable
    # YOUR CODE HERE
    
    return model

❓ **Question** ❓ We will write a function that adds flattening and dense layers after the first convolutional layers. To do so, cannot directly use the classic `layers.Sequential()` instantiation.

For that reason, we will see another one here. The idea is that we define each layer (or group of layers) separately. Then, we concatenate them. See this example : 

---
```python
base_model = load_model()
base_model = set_nontrainable_layers(base_model)
flattening_layer = layers.Flatten()
dense_layer = layers.Dense(SOME_NUMBER_1, activation='relu')
prediction_layer = layers.Dense(SOME_NUMBER_2, activation='APPROPRIATE_ACTIVATION')

model = tf.keras.Sequential([
  base_model,
  flattening_layer,
  dense_layer,
  prediction_layer
])

```
---
The first line loads a group of layer which is the previous VGG-16 model. Then, we set this layers to be non-tranable. Then, we can instantiate as many layers as we want.

Finally, we use the `Sequential` with the sequence of layers that will correspond to our overall neural network. 

Replicate the following steps by adding a flattening and two dense layers (the first with 500 neurons) to the previous VGG-16 model.

In [0]:
from tensorflow.keras import layers, models

def add_last_layers(model):
    '''Take a pre-trained model, set its parameters as non-trainables, and add additional trainable layers on top'''
    pass  # YOUR CODE HERE
    return model

❓ **Question** ❓ Now look at the layers and parameters of your model. Note that there is a distinction, at the end, between the trainable and non-trainable parameters

In [0]:
# YOUR CODE HERE

❓ **Question** ❓ Write a function that build (and compile) your model - we advise Adam with `learning_rate=1e-4`. 

In [0]:
from tensorflow.keras import optimizers

def build_model():
    pass  # YOUR CODE HERE

## 3.2 Back to the data

The VGG16 model was trained on images which were preprocessed in a specific way. This is the reason why we did not normalized them earlier.

❓ **Question** ❓ Apply this processing to the original (non-normalized) images here using the method `preprocess_input` that you can import from `tensorflow.keras.applications.vgg16`. See [documentation](https://www.tensorflow.org/api_docs/python/tf/keras/applications/vgg16/preprocess_input).

In [0]:
from tensorflow.keras.applications.vgg16 import preprocess_input

In [0]:
# YOUR CODE HERE

## 3.3 Fit the model

❓ **Question** ❓ Now estimate the model, with an early stopping criterion on the validation accuracy - here, the validation data are provided, therefore use `validation_data` instead of `validation_split`.

❗ **Remark** ❗ Store the results in a `history` variable

In [0]:
# YOUR CODE HERE

❓ **Question** ❓ Plot the accuracy for the test and validation set using the usual function below

In [0]:
def plot_history(history, title='', axs=None, exp_name=""):
    if axs is not None:
        ax1, ax2 = axs
    else:
        f, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
    
    if len(exp_name) > 0 and exp_name[0] != '_':
        exp_name = '_' + exp_name
    ax1.plot(history.history['loss'], label='train' + exp_name)
    ax1.plot(history.history['val_loss'], label='val' + exp_name)
    #ax1.set_ylim(0., 2.2)
    ax1.set_title('loss')
    ax1.legend()

    ax2.plot(history.history['accuracy'], label='train accuracy'  + exp_name)
    ax2.plot(history.history['val_accuracy'], label='val accuracy'  + exp_name)
    #ax2.set_ylim(0.25, 1.)
    ax2.set_title('Accuracy')
    ax2.legend()
    return (ax1, ax2)

In [0]:
# YOUR CODE HERE

❓ **Question** ❓ Evaluate the model accuracy on the test set. Did we improve?

In [0]:
# YOUR CODE HERE

# (OPTIONAL). Improve the model

You can here try to improve the model test accuracy. To do that, here are some options you can consider

1. **Unfreeze and finetune**: As per [Google tutorial](https://www.tensorflow.org/guide/keras/transfer_learning#fine-tuning) 
>_Once your model has converged on the new data, you can try to unfreeze all or part of the base model and retrain the whole model end-to-end with a very low learning rate. This is an optional last step that can potentially give you incremental improvements. It could also potentially lead to quick overfitting -- keep that in mind. It is critical to only do this step after the model with frozen layers has been trained to convergence. If you mix randomly-initialized trainable layers with trainable layers that hold pre-trained features, the randomly-initialized layers will cause very large gradient updates during training, which will destroy your pre-trained features. It's also critical to use a very low learning rate at this stage, because you are training a much larger model than in the first round of training, on a dataset that is typically very small. As a result, you are at risk of overfitting very quickly if you apply large weight updates. Here, you only want to readapt the pretrained weights in an incremental way._


1. Add **data augmentation** if your model is overfitting. 

1. If your model is not - unlikely here - , try a more complex model.

1. Perform precise **grid search** on all the hyper-parameters: learning_rate, batch_size, data augmentation etc...

1. **Change the base model** to more modern one (resnet, efficient net1. available in the keras library

1. Curate the data: maintaining a sane data set is one of the keys to success.

1. Obtain more data

## 🏁 Congratulation 🏁 
Copy this notebook from your google drive into your local data-challenge repo, and commit+push your progress on github. To find where this Colab notebook has been save, click on `File --> Locate in Drive`.